# My first notebook!
Execute the following line to setup C# environment


In [ ]:
dotnet tool install -g Microsoft.dotnet-interactive

### Let's try this out

👋 In the event that you hit "play" below on the code block and it asks you to:
 
```
Select kernel for <filename>
----
.NET Interactive 👈
Select Another Kernel...
----
```

Choose `.NET Interactive` and you're good to go. That selection lets you magically run the Semantic Kernel notebooks.

In [2]:
using System;
using System.IO;

string filePath = Directory.GetCurrentDirectory();
Console.WriteLine("Current file path: " + filePath);

Console.WriteLine(".NET Version: " + Environment.Version);

Current file path: c:\Users\mrochon\source\repos\ai-samples\NotebookSK\1. Start
.NET Version: 8.0.1


In [5]:
#!import ../config/Settings.cs

bool useAzureOpenAI = true;

await Settings.AskAzureEndpoint(useAzureOpenAI);
await Settings.AskModel(useAzureOpenAI);
await Settings.AskApiKey(useAzureOpenAI);

Settings: OK: Azure OpenAI endpoint configured [../config/settings.json]
Settings: OK: deployment name configured [../config/settings.json]
Settings: OK: API key configured [../config/secretsjson]


Now trying to connect to Azure Open AI

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.0.1"

In [19]:
#!import ./config/Settings.cs

using Microsoft.SemanticKernel;
using System.IO;

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();
var kernel = Microsoft.SemanticKernel.Kernel.CreateBuilder()
            .AddAzureOpenAIChatCompletion(
                 model,   // deployment name
                 azureEndpoint, // Azure OpenAI Endpoint
                 apiKey)      // Azure OpenAI Key
            .Build();

Console.WriteLine(model);

mrdeployment01


Create plugin from prompt

In [ ]:
const string prompt = @"
    ChatBot can have a conversation with you about any topic.
    It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$userInput}}
ChatBot:";

var kernelFunction = kernel.CreateFunctionFromPrompt(
    prompt,
    new Microsoft.SemanticKernel.Connectors.OpenAI.OpenAIPromptExecutionSettings
    {
        MaxTokens = 2000,
        Temperature = 0.7,
        TopP = 0.5
    });
var kernelArgs = new KernelArguments
{
    ["history"] = ""
};
Console.WriteLine("Start asking me questions...");

do
{
    var input = await InteractiveKernel.GetInputAsync("Question");
    if (input!.StartsWith("q", StringComparison.OrdinalIgnoreCase))
    {
        break;
    }
    kernelArgs["userInput"] = input;
    var answer = await kernelFunction.InvokeAsync(kernel, kernelArgs);
    var result = $"\nUser: {input}\nMelody: {answer}\n";
    kernelArgs["history"] += result;
    Console.WriteLine(result);
} while (true);


Start asking me questions...

User: How wide is the English Channel?
Melody: The English Channel is approximately 34 kilometers (21 miles) wide at its narrowest point.

